# Analyse d'un email suspect

Le vecteur initial d'intrusion le plus courant dans un incident cyber est un mail de phishing. \
C'est pour cela qu'il est très important de pouvoir analyser facilement un email reçu afin d'identifier très facilement un mail légitime d'un mail malveillant.

Il existe plusieurs format de mail ".msg" et ".eml". \
Les outils pour réaliser les analyses sont différents. Nous allons donc créer des sections distinctes.

Voici quelques ressources utilisées : 
- https://www.intezer.com/blog/incident-response/automate-analysis-phishing-email-files/


Ce playbook a pour objectif d'extraire et analyser les en-têtes des Emails pour répondre aux questions suivantes : 
- Quand a été envoyé ce mail ?
- Quelle l'@ip du serveur SMTP à l'origine de l'envoi ?
- Quelle est l'adresse email de l'expéditeur ?
- Quelle est l'adresse email de la personne destinataire du mail ?
- Est-ce que le contenu est malveillant (url, pièce jointe)?


Pour ce faire, voici les plus importants :
- Expéditeur du mail, plusieurs valeurs permettent de l'identifier. Si ces valeurs diffèrent, nous sommes très certainement face à une tentative d'usurpation d'identité.
    - **From**
    - **envelope-from**
    - **Return-Path**, **reply-to**, correspond à l'adresse mail à laquelle une réponse au mail sera envoyée;
    - **smtp.mailfrom**
    - **smtp.pra**, 
    - **x-sender**
- Informations relatives à la sécurité :
    - **x-spam status**, affiche une qualification du mail 

In [ ]:
# Load python modules
from colorama import init, Fore, Back, Style
from datetime import datetime
from defang import defang
import eml_parser
import msticpy as mp
import os
import pandas as pd
import msticpy.sectools as sectools
import re
import magic
from msg_parser import MsOxMessage
 

#Expand the width of the cells
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

!export MSTICPYCONFIG=./msticpyconfig.yaml
mp.init_notebook(globals(), verbosity=0)
ti = mp.TILookup()
ioc_extractor = sectools.IoCExtract()

# Regex 
regex_email = r"\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\b"
regex_message_id = r"[^@]+@([^'>]+)"

# Chemin du fichier à analyser
emailFile = {}
emailFile['path'] = "/home/secubian/Documents/Cases/May_God_Bless_You...eml"  

# Determine the file type

Ce controle permet d'identifier s'il s'agit d'un fichier EML ou MSG Outlook, car les outils d'analyse sont différents.

In [ ]:
import os.path
if os.path.exists(emailFile['path']):
    fileType = magic.Magic().id_filename(emailFile['path'])
    if "Microsoft Outlook Message" in fileType and ".msg" in emailFile['path']:
        print("This is a MSG file")
        emailType = "msg"
    elif "text" in fileType and ".eml" in emailFile['path']:
        print("This is an EML file")
        emailType = "eml"
    else:
        print("File Type not supported!")
else:
    print(f"File [{emailFile['path']}] doesn't exist")

## Analyse des mails au format EML

La première étape consiste à extraire les informations utiles telles que les entêtes. \
Ces informations seront placées dans des variables pour être analysées ci-dessous.

In [ ]:
mail_headers = {}
headers = {}

eml = eml_parser.parser.decode_email(emailFile['path'], include_attachment_data=True)
headers = eml['header']
mail_headers['From'] = headers['from']
mail_headers['Subject'] = headers['subject']
print(f"Le mail semble avoir été envoyé par [{mail_headers['From']}] \navec le sujet suivant : [{mail_headers['Subject']}]")

### Analyse de l'expéditeur

Il est facile de falcifier l'adresse mail à l'origine de l'envoi du mail. \
Sans rentrer dans les détails, nous allons analyser les entêtes comportant l'adresse mail de l'expéditeur. \
Si l'analyse recense plus d'une adresse mail, alors nous sommes potentiellement face à une usurpation d'identité.

In [ ]:
# Analyse senders adresses
senders_headers = ['from','envelope-from','smtp.mailfrom','smtp.pra','reply-to','return-path','x-sender']
senders_email = []

for sender in senders_headers:
    if sender in headers['header']:
        match = re.search(regex_email, headers['header'][sender][0])
        if match is not None:
            senders_email.append(match.group())

if len(senders_email) > 1: senders_email = set(senders_email)

if (mail_headers['From'] in str(senders_email)) and len(senders_email) == 1:
    print(Fore.GREEN + "[✓] No Potentially identity usurpation")
    print(Fore.GREEN + f"[✓] Only one email address has been identified : {mail_headers['From']}")
else:
    print(Fore.RED + "[!] Potentially identity usurpation during headers analysis (from, envelope-from, x-sender, smtp.mailfrom, return-path, reply-to)")
    print(f"Email addresses specified in From value : {mail_headers['From']}")
    print(f"All email addresses identified in headers : {senders_email}")

### Analyse de/des destinataire(s) du mail

En cas d'incident de sécurité, il est opportun de connaitre l'ensemble des personnes ayant reçu le mail malveillant pour réaliser le confinement de l'attaque (isolation du poste de travail, exécution d'un scan antivirus, etc...)

In [ ]:
# Analyse "to" adresses
to_headers = ['to','delivered-to','envelope-to']
to_email = []

for sender in to_headers:
    if sender in headers['header']:
        match = re.search(regex_email, headers['header'][sender][0])
        if match is not None:
            to_email.append(match.group())

if len(to_email) > 1: to_email = set(to_email)

print(f"Email has been delivered to : {to_email}")


### Analyse de l'entête "Message ID"

Il peut être observé durant l'analyse de l'entête "Message ID" un nom de domaine différent de celui présent dans l'adresse mail de l'expéditeur. Cette situation peut aider à identifier un mail malveillant.

In [ ]:
result = re.search(regex_message_id, str(headers['header']['message-id']))
message_id_domain = result.group(1)
if message_id_domain not in mail_headers['From']:
    print(Fore.RED + f"[!] Domain in Message-ID ({message_id_domain}) doesn't equal with 'From' email address domain ({mail_headers['From']})")
else:
    print(Fore.GREEN + f"[✓] Message-ID domain ({message_id_domain}) is similar with 'From' email address domain ({mail_headers['From']})")

### Analyse des serveurs ayant émis le mail
La prochaine étape consiste à identifier le serveur ayant émis le mail.

In [ ]:
regex_server_ip = r"\[(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})\]"
email_servers_ips_lst = []
for server in headers['header']['received']:
    result = re.search(regex_server_ip, server)
    if result is not None:
        email_servers_ips_lst.append(result.group(1))
if len(email_servers_ips_lst) > 0:
    print(Fore.GREEN + f"Email was sent by : {email_servers_ips_lst}")
else:
    print(Fore.RED + f"[!] There is an issue in  headers analyse.")


Une recherche en source ouverte peut maintenant avoir lieu pour identifier toute adresse ip connue et qualifiée comme malveillante.

In [ ]:
df_emails_servers_ips = pd.DataFrame(email_servers_ips_lst, columns=['servers_ips'])
df_emails_servers_ips_ti = ti.lookup_iocs(data=df_emails_servers_ips, ioc_col="servers_ips", providers=["VirusTotal", "OTX"])
df_emails_servers_ips_ti

df_emails_servers_ips_ti = pd.json_normalize(data=df_emails_servers_ips_ti[['Ioc','Provider','Details']].to_dict(orient='records'))

if (df_emails_servers_ips_ti.empty):
    print(Fore.GREEN + "[✓] No Potentially Suspicious Network connections identified")
else:
    print(Fore.RED + "[!] Potentially Suspicious Network connections identified")
    display(df_emails_servers_ips_ti)

### Analyse de la durée nécessaire à l'envoi du mail

Un mail n'a besoin que de quelques secondes pour transiter entre le client de messagerie de l'expéditeur et le serveur de messagerie du destinataire. \
Il est donc plutot facile d'identifier des mails envoyés à des groupes de diffusion, ou spam en masse qui demande plus de temps de traitement aux serveurs intermédiaires durant le transite du mail. Nous avons décidé ici d'établir un seuil à 60 secondes.

In [ ]:
from datetime import timezone
# Analyse the delta between the sending of the email by the sender and its reception by the recipient. 
mail_date_sent = headers['date'].replace(tzinfo=timezone.utc)
mail_date_received = headers['received'][0]['date'].replace(tzinfo=timezone.utc)
delta = mail_date_received - mail_date_sent

print(f"Sending date : {mail_date_sent}")
print(f"Receiving date : {mail_date_received}")

if delta.total_seconds() > 60:
    print(Fore.RED + f"[!] It seems to be a SPAM : {delta.total_seconds()} seconds ")
else:
    print(Fore.GREEN + f"[✓] It seems to be a Legit mail : {delta.total_seconds()} seconds ")
    
mail_headers['mail_time_delay (seconds)'] = delta.total_seconds()

### Extraction des pièces jointes

L'analyse du fichier EML continue en réalisant une extraction des pièces jointes. \
Ces dernières peuvent contenir du code malveillant. Il est donc nécessaire d'être vigilant.

Une des premières actions pourrait être l'analyse des pièces jointes via un antivirus, ou rechercher leur hash.

In [ ]:
import base64
outpath = f"{emailFile['path']}_attachments/"
if 'attachment' in eml:
    if len(eml['attachment']) > 0:
        for attachment in eml['attachment']:
            filename = attachment['filename']
            filename = os.path.join(outpath, filename)
            if not os.path.exists(os.path.dirname(filename)):
                try:
                    os.makedirs(os.path.dirname(filename))
                except OSError as exc:  # Guard against race condition
                    if exc.errno != errno.EEXIST:
                        raise

            print(Fore.RED + f"[!] Writing attachment: {filename}")

            with open(filename, 'wb') as a_out:
                a_out.write(base64.b64decode(attachment['raw']))
    else:
        print(Fore.GREEN + f"[✓] No attachment included")
else:
    print(Fore.GREEN + f"[✓] No attachment included")

La commande ci-dessous va générer les empreintes numériques des fichiers extraits. \
Il est tout à fait possible à l'issue de réaliser une recherche en sources ouvertes de ces valeurs (ex: https://www.virustotal.com/gui/home/search)

In [ ]:
outpath_hash = outpath.replace(' ','\ ')
!hashdeep -r $outpath_hash

## Analyse des mails au format MSG Outlook



In [ ]:
"""
#!pip3 install extract-msg
import extract_msg
import json
import re

f = emailFile['path']  # Replace with yours
msg = extract_msg.Message(f)
msg_sender = msg.sender
msg_date = msg.date
msg_subj = msg.subject
msg_message = msg.body

#print('Sender: {}'.format(msg_sender))
#print('Sent On: {}'.format(msg_date))
#print('Subject: {}'.format(msg_subj))
#print('Body: {}'.format(msg_message))
headers = json.loads(json.dumps(msg.headerDict))
#headers['Received-SPF']
#headers['User-Agent']

def runRegexList(reg, data, options=""):
    results = []
    matches = re.finditer(reg, data, options)
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(1, len(match.groups())):
            groupNum = groupNum + 1
            results.append(match.group(groupNum))
    return results

## Extraction des entêtes liées à l'expéditeur du mail
headersFrom = []
regexEmail = r"(envelope-from=.|x-sender=.|smtp.mailfrom=)([^<\s\r\n)\"':;]+@[^>\s\r\n)\"';,]+)"
#headersFrom.append(runRegex(regexEmail, str(headers['From']), re.MULTILINE))
headersFrom.append(runRegexList(regexEmail, str(msg.headerDict), re.MULTILINE))
headersFrom
"""



In [ ]:
recipient_header=['recipients']
sender_headers=['SenderEmailAddress','SenderName','SenderSearchKey','SenderSmtpAddress','SentRepresentingSmtpAddress']
subject_header=['Subject']
mail_headers = {}
headers = {}

import chardet
msg = extract_msg.openMsg(emailFile['path'])
msg_obj = MsOxMessage(emailFile['path'])

body_encoding = chardet.detect(msg.htmlBody)['encoding']
body_html = msg.htmlBody.decode(body_encoding) if msg.htmlBody else ''

headers = msg_obj.header_dict
#json_string = msg_obj.get_message_as_json()
msg_properties_dict = msg_obj.get_properties()
headers_raw = msg_properties_dict['TransportMessageHeaders']

mail_headers['From'] = headers['From'][0]
mail_headers['Subject'] = headers['Subject']
#mail_headers['Attachments'] = headers['attachments']

mail_headers['From']

### Analyse de l'expéditeur

Il est facile de falcifier l'adresse mail à l'origine de l'envoi du mail. \
Sans rentrer dans les détails, nous allons analyser les entêtes comportant l'adresse mail de l'expéditeur. \
Si l'analyse recense plus d'une adresse mail, alors nous sommes potentiellement face à une usurpation d'identité.

In [ ]:
# Analyse senders adresses
senders_email = []
regex = [
            {"name":"From (envelope-from)", "value":r'envelope-from=\"([^;]+)\"', "regexGroupID":0},
            {"name":"From (smtp.mailfrom)", "value":r'smtp.mailfrom=([^;]+)', "regexGroupID":0},
            {"name":"From (smtp.pra)", "value":r'smtp.pra=([^;]+)', "regexGroupID":0},
            {"name":"From (return-path)", "value":r'Return-Path: ([^\\\r]+)', "regexGroupID":0},
            {"name":"From (x-sender)", "value":r'X-Sender: ([^\\\r]+)', "regexGroupID":0},
        ]

for r in regex:
    result = re.search(r['value'], str(headers_raw))
    try:
        senders_email.append(result.group(1))
    except:
        pass

senders_email = list(set(senders_email))
if (mail_headers['From'] in str(senders_email)) and len(senders_email) == 1:
    print(Fore.GREEN + "[✓] No Potentially identity usurpation")
    print(Fore.GREEN + f"[✓] Only one email address has been identified : {mail_headers['From']}")
else:
    print(Fore.RED + "[!] Potentially identity usurpation during headers analysis (from, envelope-from, x-sender, smtp.mailfrom, return-path)")
    print(f"Email addresses specified in From value : {mail_headers['From']}")
    print(f"All email addresses identified in headers : {senders_email}")



### Analyse de l'entête "Message ID"

Il peut être observé durant l'analyse de l'entête "Message ID" un nom de domaine différent de celui présent dans l'adresse mail de l'expéditeur. Cette situation peut aider à identifier un mail malveillant.

In [ ]:
regex_message_id = r"Message-ID:\s<[^@]+@([^>]+)>"
result = re.search(regex_message_id, headers)
message_id_domain = result.group(1)
if message_id_domain not in mail_headers['From']:
    print(Fore.RED + f"[!] Domain in Message-ID ({message_id_domain}) doesn't equal with 'From' email address domain ({mail_headers['From']})")
else:
    print(Fore.GREEN + f"[✓] Message-ID domain ({message_id_domain}) is similar with 'From' email address domain ({mail_headers['From']})")


### Analyse de la durée nécessaire à l'envoi du mail

Un mail n'a besoin que de quelques secondes pour transiter entre le client de messagerie de l'expéditeur et le serveur de messagerie du destinataire. \
Il est donc plutot facile d'identifier des mails envoyés à des groupes de diffusion, ou spam en masse qui demande plus de temps de traitement aux serveurs intermédiaires durant le transite du mail. Nous avons décidé ici d'établir un seuil à 60 secondes.

In [ ]:
# Analyse the delta between the sending of the email by the sender and its reception by the recipient. 
regex = r"Received: from[^;]+;\s(\S{3},\s\d{0,2}\s\S{3}[^:]+\d{0,2}:\d{0,2}:\d{0,2}\s\+\d{4})"
matches = re.finditer(regex, headers, re.MULTILINE)

mail_transfert_date = []
for matchNum, match in enumerate(matches, start=1):
    for groupNum in range(0, len(match.groups())):
        groupNum = groupNum + 1        
        mail_transfert_date.append(datetime.strptime(match.group(groupNum).replace('\r\n',''), '%a, %d %b %Y %H:%M:%S %z'))

mail_transfert_date_youngest = min(mail_transfert_date)
mail_transfert_date_oldest = max(mail_transfert_date)

delta = mail_transfert_date_oldest - mail_transfert_date_youngest
if delta.total_seconds() > 60:
    print(Fore.RED + f"[!] It seems to be a SPAM : {delta.total_seconds()} seconds ")
else:
    print(Fore.GREEN + f"[✓] It seems to be a Legit mail : {delta.total_seconds()} seconds ")
    
mail_headers['mail_time_delay (seconds)'] = delta.total_seconds()